In [1]:
%matplotlib inline 
import nltk, re, numpy as np
from nltk import word_tokenize
from gensim.summarization import summarize, keywords


/Users/macbook/anaconda/lib/python2.7/site-packages/gensim-0.13.3-py2.7-macosx-10.6-x86_64.egg/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [40]:
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk.data
from bs4 import BeautifulSoup


##### webscrape the terms of service page from the url

In [151]:
import requests

raw_html = requests.get('https://www.google.com/policies/terms/').text.encode('ascii','ignore')

In [165]:
#with open('twitter_tos.htm', 'r') as f:
#    raw_html = f.read()

###### Use Beautiful Soup package to extract text from the html

In [166]:
input = BeautifulSoup(raw_html,"lxml").text

###### Use Regular expression to remove weblinks

In [167]:
import re
input = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', input)

In [168]:
input=input.encode("ascii","ignore")

In [1]:
# Simple Summarizer
# Copyright (C) 2010-2012 Tristan Havelick
# Author: Tristan Havelick <tristan@havelick.com>
# URL: <https://github.com/thavelick/summarize/>
# For license information, see LICENSE.TXT


class SimpleSummarizer:

	def reorder_sentences( self, output_sentences, input ):
		output_sentences.sort( lambda s1, s2:
			input.find(s1) - input.find(s2) )
		return output_sentences
	
	def get_summarized(self, input, num_sentences ):
		# TODO: allow the caller to specify the tokenizer they want
		# TODO: allow the user to specify the sentence tokenizer they want
		
		tokenizer = RegexpTokenizer('\w+')
		
		# get the frequency of each word in the input
		base_words = [word.lower() for word in tokenizer.tokenize(input)]
		words = [word for word in base_words if word not in stopwords.words()]
		word_frequencies = FreqDist(words)
		
		# now create a set of the most frequent words
		most_frequent_words = [pair[0] for pair in
			word_frequencies.items()[:100]]
		
		# break the input up into sentences.  working_sentences is used
		# for the analysis, but actual_sentences is used in the results
		# so capitalization will be correct.
		
		sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
		actual_sentences = sent_detector.tokenize(input)
		working_sentences = [sentence.lower() for sentence in actual_sentences]

		# iterate over the most frequent words, and add the first sentence
		# that inclues each word to the result.
		output_sentences = []

		for word in most_frequent_words:
			for i in range(0, len(working_sentences)):
				if (word in working_sentences[i]
				  and actual_sentences[i] not in output_sentences):
					output_sentences.append(actual_sentences[i])
					break
				if len(output_sentences) >= num_sentences: break
			if len(output_sentences) >= num_sentences: break
			
		# sort the output sentences back to their original order
		return self.reorder_sentences(output_sentences, input)
	
	def summarize(self, input, num_sentences):
		return self.get_summarized(input, num_sentences)

##### Create object of SimpleSummarizer Class and pass the text. Number of Output sentences as 10

In [170]:
ss=SimpleSummarizer()
summary=ss.summarize(input,10)

In [179]:
newsummary=[]
for sent in summary[1:]:
    print sent
    print

Using our Services
You must follow any policies made available to you within the Services.

In connection with your use of the Services, we may send you service announcements, administrative messages, and other information.

Do not use such Services in a way that distracts you and prevents you from obeying traffic or safety laws.

The rights you grant in this license are for the limited purpose of operating, promoting, and improving our Services, and to develop new ones.

Our automated systems analyze your content (including emails) to provide you personally relevant product features, such as customized search results, tailored advertising, and spam and malware detection.

If you have a Google Account, we may display your Profile name, Profile photo, and actions you take on Google or on third-party applications connected to your Google Account (such as +1s, reviews you write and comments you post) in our Services, including displaying in ads and other commercial contexts.

You may not 